# Hacks AI Прогноз спроса на лекарства

## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

from sklearn import linear_model

## Метрики и Лоссы

In [ ]:
def sMdAPE(y_true, y_pred):

    """
    Возвращает значение метрики sMdAPE

    Parameters
    ----------
    y_true : np.array, вектор истинных значений
    y_pred : np.array, вектор предсказанных значений
    """

    difference = np.abs(y_true - y_pred)
    sum = y_true + y_pred
    values = difference/sum
    median = np.median(values)
    return median

## Загрузка данных

In [ ]:
# данные (лекарства на складе)
df = pd.read_csv('alldata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# данные (лекарства реализованные)
results = pd.read_csv('result.csv', sep='#')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# данные по ковиду (заражения)
covid_data = pd.read_csv('covid_data.csv', sep=';')

In [ ]:
# погода + население и координаты
weather = pd.read_csv('График температур_2021-11-13_10-10.csv', sep=';')
coords = pd.read_csv('data.csv')

## DF - предобработка

Нужно смёрджить повторяющиеся колонки

In [ ]:
df['Код субъекта'].fillna(value = df['Код субъекта РФ'], inplace = True)
df.drop('Код субъекта РФ',axis = 1, inplace = True)
df['Субъект'].fillna(value = df['Наименование субъекта РФ'], inplace = True)
df.drop('Наименование субъекта РФ',axis = 1, inplace = True)
df['Масса/объем в первичной упаковке'].fillna(value = df['Масса/объем в первичной уп-ке'], inplace = True)
df.drop('Масса/объем в первичной уп-ке',axis = 1, inplace = True)
df['Количество'].fillna(value = df['Количество хранимой продукции (шт)'], inplace = True)
df.drop('Количество хранимой продукции (шт)',axis = 1, inplace = True)

df.drop(['Субъект', 'filename'], axis = 1, inplace = True)

origin = {'Отечественное':1, 'Иностранное': 2}
df['Происхождение'] = df['Происхождение'].map(origin)

In [ ]:
# выгрузим на диск
df.to_csv('alldata_merged.csv')

## Results - предобработка

Нужно смёрджить повторяющиеся колонки

In [ ]:
results = results.iloc[:-1].drop(['year_', 'name'], axis = 1) # дропаем лишние строки
results.columns = list(df.columns[:-1]) + list(results.columns[-4:]) # задаём единые названия атрибутов (как и в df)
results['Месяц'] = results['Месяц'].astype('int')

results['Количество первичной в потребительской упаковке'] = results['Количество первичной в потребительской упаковке'].apply( lambda x: np.nan if x == 'НЕ УКАЗАНО' else x)
results['Количество первичной в потребительской упаковке'] = results['Количество первичной в потребительской упаковке'].astype('float32', errors = 'ignore')

# меняем текстовые значения на численные
d = {'Нет':0, 'Да': 1}

results['ЖНЛП'] = results['ЖНЛП'].map(d)
results['Ковид'] = results['Ковид'].map(d)

## Merge results and df

In [ ]:
# делаем операцию LEFT JOIN df на results
data_merged = pd.merge(results, df, how='left', on=['Месяц',
                                                    'МНН',
                                                    'Код субъекта',
                                                    'Нормализованное наименование формы выпуска ЛП',
                                                    'Нормализованное наименование дозировки ЛП',
                                                    'Количество первичной в потребительской упаковке',
                                                    'Масса/объем в первичной упаковке',
                                                    'ЖНЛП',
                                                    'Ковид',
                                                    'Происхождение'
                                                       ])

In [ ]:
# полный совмещённый датасет (склад+релиз) - data_merged
data_merged.to_csv('data_merged.csv')

In [ ]:
# чистим память
del results
del df

gc.collect()

## Merge covid_data

In [ ]:
# словарь соответствия регионов и их номеров
dict_ = {'Адыгея': 1.0,
 'Алтай': 4.0,
 'Алтайский край': 22.0,
 'Амурская обл.': 28.0,
 'Архангельская обл.': 29.0,
 'Астраханская обл.': 30.0,
 'Башкортостан': 2.0,
 'Белгородская обл.': 31.0,
 'Брянская обл.': 32.0,
 'Бурятия': 3.0,
 'Владимирская обл.': 33.0,
 'Волгоградская обл.': 34.0,
 'Вологодская обл.': 35.0,
 'Воронежская обл.': 36.0,
 'Дагестан': 5.0,
 'Еврейская АО': 79.0,
 'Забайкальский край': 75.0,
 'Ивановская обл.': 37.0,
 'Ингушетия': 6.0,
 'Иркутская обл.': 38.0,
 'Кабардино-Балкария': 7.0,
 'Калининградская обл.': 39.0,
 'Калмыкия': 8.0,
 'Калужская обл.': 40.0,
 'Камчатский край': 41.0,
 'Карачаево-Черкессия': 9.0,
 'Карелия': 10.0,
 'Кемеровская обл.': 42.0,
 'Кировская обл.': 43.0,
 'Коми': 11.0,
 'Костромская обл.': 44.0,
 'Краснодарский край': 23.0,
 'Красноярский край': 24.0,
 'Крым': 91.0,
 'Курганская обл.': 45.0,
 'Курская обл.': 46.0,
 'Ленинградская обл.': 47.0,
 'Липецкая обл.': 48.0,
 'Магаданская обл.': 49.0,
 'Марий Эл': 12.0,
 'Мордовия': 13.0,
 'Москва': 77.0,
 'Московская обл.': 50.0,
 'Мурманская обл.': 51.0,
 'Ненецкий АО': 83.0,
 'Нижегородская обл.': 52.0,
 'Новгородская обл.': 53.0,
 'Новосибирская обл.': 54.0,
 'Омская обл.': 55.0,
 'Оренбургская обл.': 56.0,
 'Орловская обл.': 57.0,
 'Пензенская обл.': 58.0,
 'Пермский край': 59.0,
 'Приморский край': 25.0,
 'Псковская обл.': 60.0,
 'Ростовская обл.': 61.0,
 'Рязанская обл.': 62.0,
 'Самарская обл.': 63.0,
 'Санкт-Петербург': 78.0,
 'Саратовская обл.': 64.0,
 'Саха (Якутия)': 14.0,
 'Сахалинская обл.': 65.0,
 'Свердловская обл.': 66.0,
 'Севастополь': 92.0,
 'Северная Осетия': 15.0,
 'Смоленская обл.': 67.0,
 'Ставропольский край': 26.0,
 'Тамбовская обл.': 68.0,
 'Татарстан': 16.0,
 'Тверская обл.': 69.0,
 'Томская обл.': 70.0,
 'Тульская обл.': 71.0,
 'Тыва': 17.0,
 'Тюменская обл.': 72.0,
 'Удмуртия': 18.0,
 'Ульяновская обл.': 73.0,
 'ХМАО – Югра': 86.0,
 'Хабаровский край': 27.0,
 'Хакасия': 19.0,
 'Челябинская обл.': 74.0,
 'Чечня': 20.0,
 'Чувашия': 21.0,
 'Чукотский АО': 87.0,
 'Ямало-Ненецкий АО': 89.0,
 'Ярославская обл.': 76.0}

In [ ]:
# предобработка загруженного набора данных (парсинг дат, замена названий регионов на номера)
covid_data['day'] = covid_data['Дата'].map(lambda x: x.split('.')[0])
covid_data['month'] = covid_data['Дата'].map(lambda x: x.split('.')[1])
covid_data['year'] = covid_data['Дата'].map(lambda x: x.split('.')[2])

covid_data['Регион'] = covid_data['Регион'].map(dict_)

In [ ]:
# аггрегирование по месяцам
covid_yandex = pd.DataFrame([])
for year in ['2020', '2021']:
    for month in covid_data.month.unique():
        temp = covid_data[(covid_data['year'] == year) & (covid_data['month'] == month)]
        temp = temp.groupby('Регион').sum()[['Заражений за день', 'Смертей за день']]
        temp_df = pd.DataFrame([])
        temp_df['Регион'] = temp.index
        temp_df['Заражений'] = temp['Заражений за день'].values
        temp_df['Смертей'] = temp['Смертей за день'].values
        temp_df['month'] = [month] * len(temp)
        temp_df['year'] = [year] * len(temp)
        covid_yandex = pd.concat([covid_yandex, temp_df], axis=0)

In [ ]:
# переопределение типа указателей дат
covid_yandex['year'] = covid_yandex['year'].astype(int)
covid_yandex['month'] = covid_yandex['month'].astype(int)

In [ ]:
# ковидная статистика - covid_yandex
covid_yandex.to_csv('data_yandex_sumed.csv', index=False)

In [ ]:
# добавление в исходный набор данных атрибута "Год"
data_merged['Год'] = [2021]*len(data_merged)
data_merged.loc[(data_merged['Месяц']==1) | (data_merged['Месяц']==2), 'Год'] = 2020

In [ ]:
# сдвиг дат на два месяца
remonth = {-1: 11, 0: 12, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10}

data_merged['Мес_сдв'] = data_merged.Месяц.map(lambda x: x-2)
data_merged.Мес_сдв = data_merged.Мес_сдв.map(remonth)

In [ ]:
# переопределение атрибутов в исходном датасете
data_merged.columns = ['Месяц_old'] + list(data_merged.columns)[1:-1] + ['Месяц']

In [ ]:
# переопределение атрибутов в датасете с ковидом
covid_yandex.columns = ['Код субъекта', 'Заражений', 'Смертей', 'Месяц', 'Год']

In [ ]:
# делаем операцию LEFT JOIN covid_yandex на data_merged
data_covi = pd.merge(data_merged, covid_yandex, how='left', on=['Код субъекта',
                                                                'Год',
                                                                'Месяц'
                                                                ])

In [ ]:
# ковидная статистика + полный совмещённый датасет (склад+релиз) - data_covi
data_covi.to_csv('data_covi.csv', index=False)

In [ ]:
# чистим память
del data_merged
del covid_yandex
del covid_data

gc.collect()

## Merge weather

In [ ]:
# предобработка загруженного набора данных (парсинг дат, дроп лишних атрибутов, переопределение атрибутов, "цифровизация" месяцев)
weather['year'] = weather.DateTime.map(lambda x: x.split('-')[0])
weather['month'] = weather.DateTime.map(lambda x: x.split('-')[1])

weather.drop(columns=['DateTime', 'year'], inplace=True)
weather.columns=list(weather.columns[:-1]) + ['Месяц']

weather['Месяц'] = weather['Месяц'].map(lambda x: int(x))

In [ ]:
# осреднение по месяцам
weather = weather.groupby('Месяц').mean().reset_index()

In [ ]:
# переопределение атрибутов
weather.columns = ['Месяц_old'] + list(weather.columns)[1:]

In [ ]:
# делаем операцию LEFT JOIN weather на data_covi
data_cv_wt = pd.merge(data_covi, weather, how='left', on=['Месяц_old'])

In [ ]:
# ковидная статистика + полный совмещённый датасет + погода
data_cv_wt.to_csv('data_cv_wt.csv', index=False)

In [ ]:
# чистим память
del weather
del data_covi

gc.collect()

## Merge cities and population

In [ ]:
# выбор городов с наибольшим населением
huge_popu = coords.loc[coords.type == 'г', ['region', 'settlement', 'population']].groupby(['region']).max().reset_index()
huge_popu.drop(columns=['settlement'], inplace=True)

# JOIN для корректного присоединения городов к выделенным значениям наибольшего населения
huge_cities = pd.merge(huge_popu, coords[['region', 'settlement', 'population']], how='inner', on=['region', 'population'])
huge_cities.drop(columns=['population'], inplace=True)

# население суммарное по регионам
reg_popu = coords[['region', 'population']].groupby('region').sum().reset_index()

# JOIN самых густонаселённых городов и суммарного населения
reg_city_popu = pd.merge(huge_cities, reg_popu, on='region')

In [ ]:
# для Мск и СПб устанавливаем единую точку - район центральный
coords[coords['region']=='Москва'] = coords.loc[(coords['region']=='Москва')&(coords.municipality=='Центральный')]
coords[coords['region']=='Санкт-Петербург'] = coords.loc[(coords['region']=='Санкт-Петербург')&(coords.municipality=='Центральный')]

In [ ]:
# делаем выборку по городам их координатам
coords_cities = coords.loc[coords.type=='г', ['region', 'settlement', 'latitude_dd', 'longitude_dd']]

In [ ]:
# JOIN самых густонаселённых городов, суммарного населения в регионе и координат этого города
reg_city_popu_coord = pd.merge(reg_city_popu, coords_cities, on=['region', 'settlement'])

In [ ]:
# словарь наименование региона: код региона
dict_regions = {'Республика Адыгея': 1.0,
 'Республика Алтай': 4.0,
 'Алтайский край': 22.0,
 'Амурская область': 28.0,
 'Архангельская область': 29.0,
 'Астраханская область': 30.0,
 'Республика Башкортостан': 2.0,
 'Белгородская область': 31.0,
 'Брянская область': 32.0,
 'Республика Бурятия': 3.0,
 'Владимирская область': 33.0,
 'Волгоградская область': 34.0,
 'Вологодская область': 35.0,
 'Воронежская область': 36.0,
 'Республика Дагестан': 5.0,
 'Еврейская автономная область': 79.0,
 'Забайкальский край': 75.0,
 'Ивановская область': 37.0,
 'Республика Ингушетия': 6.0,
 'Иркутская область': 38.0,
 'Кабардино-Балкарская республика': 7.0,
 'Калининградская область': 39.0,
 'Республика Калмыкия': 8.0,
 'Калужская область': 40.0,
 'Камчатский край': 41.0,
 'Карачаево-Черкесская республика': 9.0,
 'Республика Карелия': 10.0,
 'Кемеровская область': 42.0,
 'Кировская область': 43.0,
 'Республика Коми': 11.0,
 'Костромская область': 44.0,
 'Краснодарский край': 23.0,
 'Красноярский край': 24.0,
 'Республика Крым': 91.0,
 'Курганская область': 45.0,
 'Курская область': 46.0,
 'Ленинградская область': 47.0,
 'Липецкая область': 48.0,
 'Магаданская область': 49.0,
 'Республика Марий Эл': 12.0,
 'Республика Мордовия': 13.0,
 'Москва': 77.0,
 'Московская область': 50.0,
 'Мурманская область': 51.0,
 'Ненецкий автономный округ': 83.0,
 'Нижегородская область': 52.0,
 'Новгородская область': 53.0,
 'Новосибирская область': 54.0,
 'Омская область': 55.0,
 'Оренбургская область': 56.0,
 'Орловская область': 57.0,
 'Пензенская область': 58.0,
 'Пермский край': 59.0,
 'Приморский край': 25.0,
 'Псковская область': 60.0,
 'Ростовская область': 61.0,
 'Рязанская область': 62.0,
 'Самарская область': 63.0,
 'Санкт-Петербург': 78.0,
 'Саратовская область': 64.0,
 'Республика Саха (Якутия)': 14.0,
 'Сахалинская область': 65.0,
 'Свердловская область': 66.0,
 'Севастополь': 92.0,
 'Республика Северная Осетия - Алания': 15.0,
 'Смоленская область': 67.0,
 'Ставропольский край': 26.0,
 'Тамбовская область': 68.0,
 'Республика Татарстан': 16.0,
 'Тверская область': 69.0,
 'Томская область': 70.0,
 'Тульская область': 71.0,
 'Республика Тыва': 17.0,
 'Тюменская область': 72.0,
 'Удмуртская республика': 18.0,
 'Ульяновская область': 73.0,
 'Ханты-Мансийский автономный округ - Югра': 86.0,
 'Хабаровский край': 27.0,
 'Республика Хакасия': 19.0,
 'Челябинская область': 74.0,
 'Чеченская республика': 20.0,
 'Чувашская республика': 21.0,
 'Чукотский автономный округ': 87.0,
 'Ямало-Ненецкий автономный округ': 89.0,
 'Ярославская область': 76.0}

In [ ]:
# предобрабатываем датасет с регионами и координатами (названия заменяем на номера, дропаем пустые строки, переопределяем атриубуты)
reg_city_popu_coord.region = reg_city_popu_coord.region.map(dict_regions)
reg_city_popu_coord.dropna(inplace=True)
reg_city_popu_coord.columns = ['Код субъекта']+list(reg_city_popu_coord.columns[1:])

In [ ]:
# получаем финальный датасет: рлиз+склад+ковид+погода+координаты+население
data_cv_wt_crd = pd.merge(data_cv_wt, reg_city_popu_coord, how='left', on=['Код субъекта'])

In [ ]:
# сохраняем финальный датасет
data_cv_wt_crd.to_csv('data_cv_wt_crd.csv', index=False)

In [ ]:
# чистим память
del data_cv_wt
del reg_city_popu_coord
del reg_city_popu
del coords_cities
del coords
del reg_popu
del huge_cities
del huge_popu

gc.collect()